### Please apply following rules: (Indicator - BUY, CALL, LONG)
    * Make sure the red line of MACD is above - 0
    * Make a bollinger band of simple moveing average - with 10 days MA and 1 as STD DEV
    * (T - 1) Yesterday's High of the stock is below the lower bound of Bollinger's Band
    * T open should not be above T -1 high
    * The stock's in T should cross T - 1's high, then buy it at T-1's high and put a stop loss of T-1's low
    * If the stock doesnt cross T-1's high in T, then dont buy it.
    * when the next price drop happens (Red candle day), but is above the stop loss; revise the stop loss 
        to the low of that day
    

### Calculation of Quantity and PnL

number of shares to buy: lpt(loss per trade)/(600-585)

ltp = 1000 (By Appetite)
    
number of share to buy = 1000/15 

buy price - initial stop loss = (600 - 585)  = (high of T-1  - low of T-1)

buy price = 600

sell price = 617

profit per share = 17

profit % = 17/600 = 2.8%

investment = 65 * 600 ~ 3900

net profit ~ 750 

In [43]:
#invite people for the Kaggle party
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import sys
# sys.path.append('./python_code')
# import featexp as fe
from sklearn.model_selection import train_test_split
from pandas_datareader import data as web 
import requests as req 
import pandas as pd
import StringIO as strio
import datetime as dt
from tqdm import tqdm_notebook as tqdm
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook, show
output_notebook()
from bokeh.models import ColumnDataSource, HoverTool
from bokeh.layouts import column
from joblib import Parallel, delayed, parallel_backend

Loading BokehJS ...

In [2]:
url = "https://www.nseindia.com/content/indices/ind_nifty50list.csv"
csvfromurl = req.get(url).content
csvfordf = pd.read_csv(strio.StringIO(csvfromurl.decode('utf-8')))
nifty50fd = pd.DataFrame(csvfordf)
nifty50fd

,Company Name,Industry,Symbol,Series,ISIN Code
0,Adani Ports and Special Economic Zone Ltd.,SERVICES,ADANIPORTS,EQ,INE742F01042
1,Asian Paints Ltd.,CONSUMER GOODS,ASIANPAINT,EQ,INE021A01026
2,Axis Bank Ltd.,FINANCIAL SERVICES,AXISBANK,EQ,INE238A01034
3,Bajaj Auto Ltd.,AUTOMOBILE,BAJAJ-AUTO,EQ,INE917I01010
4,Bajaj Finance Ltd.,FINANCIAL SERVICES,BAJFINANCE,EQ,INE296A01024
5,Bajaj Finserv Ltd.,FINANCIAL SERVICES,BAJAJFINSV,EQ,INE918I01018
6,Bharat Petroleum Corporation Ltd.,ENERGY,BPCL,EQ,INE029A01011
7,Bharti Airtel Ltd.,TELECOM,BHARTIARTL,EQ,INE397D01024
8,Bharti Infratel Ltd.,TELECOM,INFRATEL,EQ,INE121J01017
9,Britannia Industries Ltd.,CONSUMER GOODS,BRITANNIA,EQ,INE216A01030


In [37]:
backtestdt = dt.date.today()
startdate = dt.date(2019, 1, 1)
enddate = dt.date(2019, 4, 23)

In [3]:
%%time

stock_df = pd.DataFrame()

diffyears = 0.0
diffyears = enddate.year - startdate.year
for eachscrip in tqdm(nifty50fd['Symbol']):
    stock = eachscrip+'.NS'
    try:
#         print "Processing:", stock
        stockdata = web.DataReader(stock, data_source='yahoo',start=startdate, end=enddate)
        stockdata['Symbol'] = eachscrip
        stock_df = stock_df.append(stockdata)
    except:
        print "Error Processing:", stock
            
print stock_df.shape


(2998, 7)
CPU times: user 6.95 s, sys: 342 ms, total: 7.29 s
Wall time: 1min 28s


In [111]:
def ExpMovingAverage(values, window):
    """ Numpy implementation of EMA
    """
    weights = np.exp(np.linspace(-1., 0., window))
    weights /= weights.sum()
    a =  np.convolve(values, weights, mode='full')[:len(values)]
    a[:window] = a[window]
    return a

In [112]:
# MACD 12, 26, 9
def rolling_mean(df_input):
    df2 = df_input.Close.rolling(window=10).agg({"cls_mean": np.mean, "cls_std": np.std})
    df2 = df2.fillna(method='bfill')
    df2 = pd.concat([df_input, df2], axis=1)
    df2['lower'] = df2.cls_mean - df2.cls_std
    df2['upper'] = df2.cls_mean + df2.cls_std
    df2['26ema'] = ExpMovingAverage(df_input.Close, 26)
    df2['12ema'] = ExpMovingAverage(df_input.Close, 12)
    df2['MACD'] = (df2['12ema'] - df2['26ema'])
    df2['Signal'] = ExpMovingAverage(df2['MACD'], 9)
    
    return df2

In [8]:
for eachscrip in tqdm(nifty50fd['Symbol']):
    stock = eachscrip
    try:
        stock_df[stock_df.Symbol == stock].Close.rolling(window=10).agg({"cls_mean": np.mean, "cls_std": np.std})
        ExpMovingAverage(stock_df[stock_df.Symbol == stock].Close, 12)
    except:
        print "error processing", stock
        stock_df = stock_df[stock_df.Symbol != stock]
        
df = stock_df

df = df.reset_index()

df = df.groupby('Symbol').apply(rolling_mean)
df_transformed = df.reset_index()

# Look at End Date
df_transformed = df_transformed[df_transformed.Date== enddate] 

# Check MACD Signal > 0 For Buy oppertunities
df_transformed = df_transformed[df_transformed.Signal > 0]

# Check if the Days High is below lower bound of MACD
final_df = df_transformed[df_transformed.High < df_transformed.lower]
final_df

,index,Date,High,Low,Open,Close,Volume,Adj Close,Symbol,cls_mean,cls_std,lower,upper,26ema,12ema,MACD,Signal


-----------

### Analyze all stocks

In [ ]:
startdate = dt.date(2019, 1, 1)
enddate = dt.date(2019, 4, 24)

In [30]:
url = r'/Users/apple/Downloads/Consolidated list of Scrips - Blocked_Allowed by RMS.xlsx'
stocks = pd.read_excel(url)['Stocks allowed for MIS']
stocks = [str(s)[:-3] for s in stocks]
stocks = np.unique(stocks)
stocks = stocks[1:]
stocks[:5]

array(['3MINDIA', '63MOONS', 'AARTIIND', 'ABB', 'ABFRL'], dtype='|S10')

In [109]:
def load_data(eachscrip):
   
    try:
#         print "Processing:", eachscrip
        stockdata = web.DataReader(eachscrip, data_source='yahoo',start=startdate, end=enddate)
        stockdata['Symbol'] = eachscrip
        return stockdata
        
    except:
        print "Error Processing:", eachscrip

In [124]:
df = Parallel(n_jobs=8)(delayed(load_data)(stock+'.NS') for stock in tqdm(stocks))

stock_df = pd.DataFrame()
for stock in df:
    stock_df= stock_df.append(stock)
stock_df.tail()

,High,Low,Open,Close,Volume,Adj Close,Symbol
Date,,,,,,,
2019-04-15,421.549988,415.000000,418.0,416.600006,2724082.0,416.600006,ZEEL.NS
2019-04-16,424.500000,414.399994,418.0,415.549988,3495678.0,415.549988,ZEEL.NS
2019-04-18,418.000000,401.000000,418.0,402.299988,4424726.0,402.299988,ZEEL.NS
2019-04-22,405.700012,394.500000,402.5,396.549988,3418483.0,396.549988,ZEEL.NS
2019-04-23,422.000000,400.000000,400.0,409.149994,9490144.0,409.149994,ZEEL.NS


In [125]:
df = stock_df
for stock in tqdm(stocks):
    stock = stock+'.NS'
    try:
        df[df.Symbol == stock].Close.rolling(window=10).agg({"cls_mean": np.mean, "cls_std": np.std})
        ExpMovingAverage(df[df.Symbol == stock].Close, 12)
    except:
        print "error processing", stock
        df = df[df.Symbol != stock]
        
df = df.reset_index()

df = df.groupby('Symbol').apply(rolling_mean)
df_transformed = df.reset_index()

# Look at End Date
df_transformed = df_transformed[df_transformed.Date == enddate] 

# Check MACD Signal > 0 For Buy oppertunities
df_transformed = df_transformed[df_transformed.Signal > 0]

# Check if the Days High is below lower bound of MACD
final_df = df_transformed[df_transformed.High < df_transformed.lower]
final_df

error processing 63MOONS.NS
error processing AVANTIFEED.NS
error processing GNFC.NS
error processing GUJGASLTD.NS
error processing JAYSREETEA.NS
error processing LINCOLN.NS
error processing M50.NS
error processing MAJESCO.NS
error processing MCDOWELL-N.NS
error processing SALZERELEC.NS
error processing SHARDAMOTR.NS
error processing SPTL.NS
error processing TIFIN.NS


,index,Date,High,Low,Open,Close,Volume,Adj Close,Symbol,cls_mean,cls_std,lower,upper,26ema,12ema,MACD,Signal
524,524,2019-04-23,136.300003,131.550003,135.399994,132.500000,4084835.0,132.500000,ADANIENT.NS,140.545000,4.050409,136.494591,144.595408,142.429612,142.764373,0.334761,1.773605
3132,3132,2019-04-23,93.750000,91.250000,91.900002,91.650002,6059692.0,91.650002,BANKINDIA.NS,97.910001,3.382701,94.527300,101.292702,98.831871,99.144351,0.312480,4.845600
3357,3357,2019-04-23,89.849998,87.900002,89.300003,88.400002,8118248.0,88.400002,BEL.NS,93.305001,2.540718,90.764283,95.845719,94.191871,94.620434,0.428562,2.066530
3432,3432,2019-04-23,926.200012,880.500000,918.000000,901.549988,3198314.0,901.549988,BEML.NS,956.750000,24.613768,932.136232,981.363768,976.292814,967.091530,-9.201284,24.288398
3582,3582,2019-04-23,196.149994,190.350006,194.699997,192.649994,244442.0,192.649994,BFUTILITIE.NS,207.989999,8.812485,199.177515,216.802484,199.600801,206.425856,6.825054,3.697607
4557,4557,2019-04-23,333.399994,327.850006,332.000000,328.750000,512591.0,328.750000,CADILAHC.NS,338.930002,5.467133,333.462869,344.397135,337.715279,341.227152,3.511873,4.599718
4782,4782,2019-04-23,322.149994,311.049988,322.149994,317.299988,1770616.0,317.299988,CANFINHOME.NS,343.144995,14.582190,328.562805,357.727185,335.368078,348.012770,12.644692,23.063292
5532,5532,2019-04-23,173.300003,167.250000,168.000000,168.350006,752343.0,168.350006,CGCL.NS,181.010001,6.291651,174.718349,187.301652,164.313360,179.607014,15.293654,17.654114
7332,7332,2019-04-23,245.699997,239.000000,239.000000,244.500000,517305.0,244.500000,DELTACORP.NS,254.545000,7.651743,246.893257,262.196743,252.760031,258.218072,5.458041,6.683013
7557,7557,2019-04-23,148.600006,136.300003,139.000000,140.149994,32161660.0,140.149994,DHFL.NS,161.144998,11.947162,149.197837,173.092160,144.940509,161.134436,16.193926,11.040069


----------

In [ ]:
source = ColumnDataSource(df.reset_index())

TOOLS = "pan,wheel_zoom,box_zoom,reset,save"
p = figure(tools=TOOLS)

p.scatter(x='Date', y='Close', line_color=None, fill_alpha=0.3, size=5, source=source)

# band = Band(base='x', lower='lower', upper='upper', source=source, level='underlay',
#             fill_alpha=1.0, line_width=1, line_color='black')
# p.add_layout(band)

p.title.text = "Rolling Standard Deviation"
p.xgrid[0].grid_line_color=None
p.ygrid[0].grid_line_alpha=0.5
p.xaxis.axis_label = 'X'
p.yaxis.axis_label = 'Y'

show(p)